In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objects as go
import json
import base64
import io
from dash_table import DataTable

# Load the dataset from Excel file with error handling
try:
    df = pd.read_excel(r'C:\Users\Abraha\Desktop\Incidents\Incident Anlalysis Dashboard Enhanced Version\cleaned_month_shortened.xlsx')
except FileNotFoundError:
    print("Error: Excel file not found at the specified path")
    exit(1)

# Remove rows where 'Short_Location', 'Month', or 'Short_Category' are null
df = df.dropna(subset=['Short_Location', 'Month', 'Short_Category'])

# Ensure the 'Month' column is in datetime format for proper sorting
df['Month'] = pd.to_datetime(df['Month'], format='%b %y')
sorted_months = sorted(df['Month'].unique())
df['Month'] = df['Month'].dt.strftime('%b %y')

# Columns to display in the incident details
detail_columns = [
    'Incident Number', 'Priority', 'Opened', 'Created', 'Closed', 'Assignment group', 'Category', 
    'Service', 'Issue', 'Opened by', 'Assigned to', 'Description', 'Response SLA Status', 
    'Resolution SLA Status', 'Reopen count', 'Short description', 'Re - Opened', 'On hold reason', 
    'Reassignment count', 'Trigger Rule', 'Resolved', 'Channel', 'Caller', 'Location', 'Month', 
    'Reopened Actual'
]

# Initialize the Dash app
app = dash.Dash(__name__)

# Add custom CSS
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .fade-in { animation: fadeIn 0.5s ease-in; }
            .slide-in { animation: slideIn 0.5s ease-out; }
            @keyframes fadeIn { 0% { opacity: 0; } 100% { opacity: 1; } }
            @keyframes slideIn { 0% { transform: translateY(20px); opacity: 0; } 100% { transform: translateY(0); opacity: 1; } }
            .heatmap-container > div { transition: all 0.3s ease; }
            .button-hover:hover { background-color: #5D8296 !important; transform: scale(1.05); transition: all 0.2s ease; }
            .remove-button-hover:hover { background-color: #E55A5A !important; transform: scale(1.05); transition: all 0.2s ease; }
            .dash-table-container { margin-top: 20px; }
            .dash-table-container .dash-spreadsheet-container .dash-spreadsheet-inner th { background-color: #34495E; color: white; }
            .dash-table-container .dash-spreadsheet-container .dash-spreadsheet-inner tr:nth-child(even) { background-color: #F7F9FA; }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

label_mapping = {
    'Short_Assignment Group': 'Assignment Group',
    'Short_Category': 'Category',
    'Short_Service': 'Service',
    'Short_Issue': 'Issue',
    'Short_Location': 'Location',
    'Month': 'Month'
}

heatmap_fields = [
    {'label': 'Assignment Group', 'value': 'Short_Assignment Group'},
    {'label': 'Category', 'value': 'Short_Category'},
    {'label': 'Service', 'value': 'Short_Service'},
    {'label': 'Issue', 'value': 'Short_Issue'},
    {'label': 'Location', 'value': 'Short_Location'},
    {'label': 'Month', 'value': 'Month'}
]

# Define the layout
app.layout = html.Div([
    dcc.Store(id='heatmap-config-store', data=[]),
    dcc.Store(id='selected-cell-store', data={}),
    html.Div([
        html.H1(
            'Incidents Analysis Dashboard (Sep 2024 - Feb 2025)',
            className='slide-in',
            style={
                'textAlign': 'center',
                'fontFamily': 'Arial, sans-serif',
                'fontSize': '40px',
                'fontWeight': 'bold',
                'color': '#FFFFFF',
                'margin': '20px 0',
                'textShadow': '2px 2px 6px rgba(0, 0, 0, 0.3)'
            }
        ),
        html.Div([
            html.Div([
                html.Label(label_mapping['Short_Assignment Group'], style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='assignment_group_dropdown',
                    options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Assignment Group'].unique().tolist()],
                    multi=True,
                    placeholder="Select Assignment Group",
                    value=[],
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '16%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label(label_mapping['Short_Category'], style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='category_dropdown',
                    options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Category'].unique().tolist()],
                    multi=True,
                    placeholder="Select Category",
                    value=[],
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '16%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label(label_mapping['Short_Service'], style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='service_dropdown',
                    options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Service'].unique().tolist()],
                    multi=True,
                    placeholder="Select Service",
                    value=[],
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '16%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label(label_mapping['Short_Issue'], style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='issue_dropdown',
                    options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Issue'].unique().tolist()],
                    multi=True,
                    placeholder="Select Issue",
                    value=[],
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '16%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label(label_mapping['Short_Location'], style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='location_dropdown',
                    options=[{'label': i, 'value': i} for i in ['All'] + df['Short_Location'].unique().tolist()],
                    multi=True,
                    placeholder="Select Location",
                    value=[],
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '16%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label('Month', style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='month_dropdown',
                    options=[{'label': i.strftime('%b %y'), 'value': i.strftime('%b %y')} for i in sorted_months],
                    multi=True,
                    placeholder="Select Month",
                    value=[],
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '16%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
        ], style={
            'display': 'flex',
            'flexWrap': 'nowrap',
            'justifyContent': 'space-between',
            'padding': '0 20px 20px 20px',
            'width': '100%',
            'boxSizing': 'border-box'
        }),
        html.Div([
            html.Div([
                html.Label('X-Axis', style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='x_axis_dropdown',
                    options=heatmap_fields,
                    value='Short_Service',
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '30%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label('Y-Axis', style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Dropdown(
                    id='y_axis_dropdown',
                    options=heatmap_fields,
                    value='Short_Issue',
                    style={'backgroundColor': '#FFFFFF', 'borderRadius': '8px', 'border': '1px solid #E0E0E0'}
                ),
            ], style={'width': '30%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
            html.Div([
                html.Label('', style={'color': '#FFFFFF', 'fontWeight': 'bold', 'marginBottom': '5px'}),
                html.Button('Generate Heatmap', id='Generate-heatmap-btn', n_clicks=0,
                           className='button-hover',
                           style={'width': '100%', 'padding': '12px', 'backgroundColor': '#7A9DAE',
                                  'color': 'white', 'border': 'none', 'borderRadius': '8px', 'cursor': 'pointer',
                                  'fontSize': '16px', 'fontFamily': 'Arial, sans-serif', 'marginTop': '2px'})
            ], style={'width': '30%', 'padding': '10px', 'display': 'inline-block', 'verticalAlign': 'top'}),
        ], style={
            'display': 'flex',
            'flexWrap': 'nowrap',
            'justifyContent': 'center',
            'padding': '0 20px 20px 20px',
            'width': '100%',
            'boxSizing': 'border-box'
        })
    ], style={
        'backgroundColor': '#34495E',
        'boxShadow': '0 6px 20px rgba(0, 0, 0, 0.25)',
        'borderRadius': '12px',
        'margin': '20px',
    }),
    html.Div(id='heatmap-container', className='heatmap-container',
             style={'marginLeft': '10%', 'marginRight': '10%', 'marginBottom': '30px'}),
    html.Div([
        html.Button('Show Details', id='show-details-btn', n_clicks=0,
                    className='button-hover',
                    style={'margin': '20px 0', 'padding': '10px 20px', 'backgroundColor': '#7A9DAE',
                           'color': 'white', 'border': 'none', 'borderRadius': '8px', 'cursor': 'pointer',
                           'fontSize': '16px', 'fontFamily': 'Arial, sans-serif'}),
        dcc.Download(id='download-csv'),
        html.Button('Export to CSV', id='export-csv-btn', n_clicks=0,
                    className='button-hover',
                    style={'margin': '20px 20px', 'padding': '10px 20px', 'backgroundColor': '#2ECC71',
                           'color': 'white', 'border': 'none', 'borderRadius': '8px', 'cursor': 'pointer',
                           'fontSize': '16px', 'fontFamily': 'Arial, sans-serif'}),
        html.Div(id='details-container', className='fade-in',
                 style={'marginBottom': '30px', 'backgroundColor': '#FFFFFF', 'padding': '20px', 
                        'borderRadius': '12px', 'boxShadow': '0 4px 15px rgba(0, 0, 0, 0.1)', 'display': 'none'})
    ], style={'marginLeft': '10%', 'marginRight': '10%', 'textAlign': 'center'})
], style={
    'backgroundColor': '#ECF0F1',
    'minHeight': '100vh',
})

# Callback to update heatmap configurations (Add)
@app.callback(
    Output('heatmap-config-store', 'data', allow_duplicate=True),
    Input('Generate-heatmap-btn', 'n_clicks'),
    State('x_axis_dropdown', 'value'),
    State('y_axis_dropdown', 'value'),
    State('heatmap-config-store', 'data'),
    prevent_initial_call=True
)
def update_heatmap_configs_add(n_clicks, x_axis, y_axis, current_configs):
    if n_clicks > 0 and x_axis and y_axis:
        new_config = {'x_axis': x_axis, 'y_axis': y_axis, 'id': f'heatmap-{n_clicks}'}
        current_configs = current_configs if current_configs else []
        if not any(config['x_axis'] == x_axis and config['y_axis'] == y_axis for config in current_configs):
            current_configs.append(new_config)
    return current_configs

# Callback to update heatmap configurations (Remove)
@app.callback(
    Output('heatmap-config-store', 'data', allow_duplicate=True),
    Input({'type': 'remove-heatmap-btn', 'index': dash.dependencies.ALL}, 'n_clicks'),
    State('heatmap-config-store', 'data'),
    prevent_initial_call=True
)
def update_heatmap_configs_remove(remove_clicks, current_configs):
    if not any(remove_clicks):
        return current_configs
    
    ctx = dash.callback_context
    if not ctx.triggered:
        return current_configs
    
    triggered_id = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])
    heatmap_id = triggered_id['index']
    updated_configs = [config for config in current_configs if config['id'] != heatmap_id]
    return updated_configs

# Callback to update heatmap container and capture cell clicks
@app.callback(
    [Output('heatmap-container', 'children'),
     Output('selected-cell-store', 'data')],
    [Input('heatmap-config-store', 'data'),
     Input('assignment_group_dropdown', 'value'),
     Input('category_dropdown', 'value'),
     Input('service_dropdown', 'value'),
     Input('issue_dropdown', 'value'),
     Input('location_dropdown', 'value'),
     Input('month_dropdown', 'value')] +
    [Input({'type': 'heatmap', 'index': dash.dependencies.ALL}, 'clickData')],
    [State('selected-cell-store', 'data')]
)
def update_heatmap_container(configs, assignment_group, category, service, issue, location, month, *args):
    click_data_list = args[:-1]  # All clickData inputs
    current_selected_cell = args[-1]  # State of selected-cell-store

    if not configs:
        return html.Div("No heatmaps generated yet. Select X and Y axes and click 'Generate Heatmap'.",
                       className='fade-in',
                       style={'textAlign': 'center', 'padding': '20px', 'color': '#34495E', 'fontSize': '18px'}), current_selected_cell

    filtered_df = df.copy()
    filters_applied = []
    if assignment_group and 'All' not in assignment_group:
        filtered_df = filtered_df[filtered_df['Short_Assignment Group'].isin(assignment_group)]
        filters_applied.append(f"Assignment Group: {', '.join(assignment_group)}")
    if category and 'All' not in category:
        filtered_df = filtered_df[filtered_df['Short_Category'].isin(category)]
        filters_applied.append(f"Category: {', '.join(category)}")
    if service and 'All' not in service:
        filtered_df = filtered_df[filtered_df['Short_Service'].isin(service)]
        filters_applied.append(f"Service: {', '.join(service)}")
    if issue and 'All' not in issue:
        filtered_df = filtered_df[filtered_df['Short_Issue'].isin(issue)]
        filters_applied.append(f"Issue: {', '.join(issue)}")
    if location and 'All' not in location:
        filtered_df = filtered_df[filtered_df['Short_Location'].isin(location)]
        filters_applied.append(f"Location: {', '.join(location)}")
    if month:
        filtered_df = filtered_df[filtered_df['Month'].isin(month)]
        filters_applied.append(f"Month: {', '.join(month)}")

    filter_suffix = " - " + " | ".join(filters_applied) if filters_applied else ""

    heatmap_components = []
    selected_cell = current_selected_cell

    ctx = dash.callback_context
    if ctx.triggered and 'clickData' in ctx.triggered[0]['prop_id']:
        triggered_id = json.loads(ctx.triggered[0]['prop_id'].split('.')[0])
        heatmap_id = triggered_id['index']
        click_data = ctx.triggered[0]['value']
        if click_data:
            point = click_data['points'][0]
            selected_cell = {
                'heatmap_id': heatmap_id,
                'x': point['x'],
                'y': point['y']
            }

    for i, config in enumerate(configs):
        x_axis = config['x_axis']
        y_axis = config['y_axis']
        heatmap_id = config['id']

        title = f"{label_mapping.get(x_axis, x_axis)} vs {label_mapping.get(y_axis, y_axis)} Heatmap{filter_suffix}"

        if filtered_df.empty:
            fig = go.Figure(
                data=[],
                layout={
                    'title': {'text': title, 'font': {'size': 20, 'color': '#34495E'}},
                    'annotations': [{'text': 'No data available', 'showarrow': False, 'font': {'size': 16}}]
                }
            )
        else:
            heatmap_data = filtered_df.groupby([x_axis, y_axis]).size().reset_index(name='Count')
            if x_axis == 'Month':
                heatmap_data['Month'] = pd.to_datetime(heatmap_data['Month'], format='%b %y')
                heatmap_data = heatmap_data.sort_values('Month')
                heatmap_data['Month'] = heatmap_data['Month'].dt.strftime('%b %y')
            if y_axis == 'Month':
                heatmap_data['Month'] = pd.to_datetime(heatmap_data['Month'], format='%b %y')
                heatmap_data = heatmap_data.sort_values('Month')
                heatmap_data['Month'] = heatmap_data['Month'].dt.strftime('%b %y')

            fig = go.Figure(data=go.Heatmap(
                x=heatmap_data[x_axis],
                y=heatmap_data[y_axis],
                z=heatmap_data['Count'],
                colorscale='YlGnBu',
                text=heatmap_data['Count'],
                texttemplate='%{text}',
                hovertemplate=f'<b>{label_mapping.get(x_axis, x_axis)}:</b> %{{x}}<br><b>{label_mapping.get(y_axis, y_axis)}:</b> %{{y}}<br><b>Count:</b> %{{z}}',
                colorbar={'title': 'Count'},
                showscale=True,
                hoverongaps=False,
                name=''
            ))
            fig.update_layout(
                title={
                    'text': title,
                    'y': 0.95,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {'size': 20, 'color': '#34495E', 'family': 'Arial, sans-serif'}
                },
                xaxis_title=label_mapping.get(x_axis, x_axis),
                yaxis_title=label_mapping.get(y_axis, y_axis),
                font=dict(family='Arial, sans-serif', size=14, color='#34495E'),
                plot_bgcolor='#FFFFFF',
                paper_bgcolor='#F7F9FA'
            )

        heatmap_components.append(
            html.Div([
                dcc.Loading(children=[dcc.Graph(figure=fig, id={'type': 'heatmap', 'index': heatmap_id}, className='fade-in')]),
                html.Button('Remove', id={'type': 'remove-heatmap-btn', 'index': heatmap_id}, n_clicks=0,
                           className='remove-button-hover',
                           style={'marginTop': '15px', 'padding': '8px 15px', 'backgroundColor': '#FF6B6B',
                                  'color': 'white', 'border': 'none', 'borderRadius': '8px', 'cursor': 'pointer',
                                  'fontSize': '14px', 'fontFamily': 'Arial, sans-serif'})
            ], style={'marginBottom': '40px', 'borderBottom': '2px dashed #BDC3C7' if heatmap_id != configs[-1]['id'] else 'none',
                      'backgroundColor': '#FFFFFF', 'padding': '20px', 'borderRadius': '12px', 'boxShadow': '0 4px 15px rgba(0, 0, 0, 0.1)'})
        )

    return heatmap_components, selected_cell

# Callback to show incident details in a paginated table
@app.callback(
    [Output('details-container', 'children'),
     Output('details-container', 'style')],
    [Input('show-details-btn', 'n_clicks'),
     Input('selected-cell-store', 'data')],
    [State('assignment_group_dropdown', 'value'),
     State('category_dropdown', 'value'),
     State('service_dropdown', 'value'),
     State('issue_dropdown', 'value'),
     State('location_dropdown', 'value'),
     State('month_dropdown', 'value'),
     State('heatmap-config-store', 'data')]
)
def show_incident_details(n_clicks, selected_cell, assignment_group, category, service, issue, location, month, heatmap_configs):
    if not selected_cell or 'heatmap_id' not in selected_cell:
        return "Click a heatmap cell and then 'Show Details' to view incident details.", {
            'display': 'block', 'marginBottom': '30px', 'backgroundColor': '#FFFFFF', 'padding': '20px', 
            'borderRadius': '12px', 'boxShadow': '0 4px 15px rgba(0, 0, 0, 0.1)'
        }

    filtered_df = df.copy()
    if assignment_group and 'All' not in assignment_group:
        filtered_df = filtered_df[filtered_df['Short_Assignment Group'].isin(assignment_group)]
    if category and 'All' not in category:
        filtered_df = filtered_df[filtered_df['Short_Category'].isin(category)]
    if service and 'All' not in service:
        filtered_df = filtered_df[filtered_df['Short_Service'].isin(service)]
    if issue and 'All' not in issue:
        filtered_df = filtered_df[filtered_df['Short_Issue'].isin(issue)]
    if location and 'All' not in location:
        filtered_df = filtered_df[filtered_df['Short_Location'].isin(location)]
    if month:
        filtered_df = filtered_df[filtered_df['Month'].isin(month)]

    config = next((c for c in heatmap_configs if c['id'] == selected_cell['heatmap_id']), None)
    if not config:
        return "Heatmap configuration not found.", {
            'display': 'block', 'marginBottom': '30px', 'backgroundColor': '#FFFFFF', 'padding': '20px', 
            'borderRadius': '12px', 'boxShadow': '0 4px 15px rgba(0, 0, 0, 0.1)'
        }
    
    x_axis = config['x_axis']
    y_axis = config['y_axis']

    detailed_df = filtered_df[(filtered_df[x_axis] == selected_cell['x']) & (filtered_df[y_axis] == selected_cell['y'])]
    if detailed_df.empty:
        return "No incidents found for this selection.", {
            'display': 'block', 'marginBottom': '30px', 'backgroundColor': '#FFFFFF', 'padding': '20px', 
            'borderRadius': '12px', 'boxShadow': '0 4px 15px rgba(0, 0, 0, 0.1)'
        }

    # Convert DataFrame to list of dictionaries for DataTable
    data = detailed_df[detail_columns].to_dict('records')
    
    # Create paginated DataTable
    table = DataTable(
        id='incident-details-table',
        columns=[{"name": col, "id": col} for col in detail_columns],
        data=data,
        page_size=50,  # 50 incidents per page
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '8px', 'fontSize': '14px', 'fontFamily': 'Arial, sans-serif'},
        style_header={'backgroundColor': '#34495E', 'color': 'white', 'fontWeight': 'bold'},
        style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#F7F9FA'}]
    )

    return table, {
        'display': 'block', 'marginBottom': '30px', 'backgroundColor': '#FFFFFF', 'padding': '20px', 
        'borderRadius': '12px', 'boxShadow': '0 4px 15px rgba(0, 0, 0, 0.1)'
    }

# Callback to export filtered data to CSV
@app.callback(
    Output('download-csv', 'data'),
    Input('export-csv-btn', 'n_clicks'),
    [State('assignment_group_dropdown', 'value'),
     State('category_dropdown', 'value'),
     State('service_dropdown', 'value'),
     State('issue_dropdown', 'value'),
     State('location_dropdown', 'value'),
     State('month_dropdown', 'value'),
     State('selected-cell-store', 'data'),
     State('heatmap-config-store', 'data')],
    prevent_initial_call=True
)
def export_to_csv(n_clicks, assignment_group, category, service, issue, location, month, selected_cell, heatmap_configs):
    filtered_df = df.copy()
    if assignment_group and 'All' not in assignment_group:
        filtered_df = filtered_df[filtered_df['Short_Assignment Group'].isin(assignment_group)]
    if category and 'All' not in category:
        filtered_df = filtered_df[filtered_df['Short_Category'].isin(category)]
    if service and 'All' not in service:
        filtered_df = filtered_df[filtered_df['Short_Service'].isin(service)]
    if issue and 'All' not in issue:
        filtered_df = filtered_df[filtered_df['Short_Issue'].isin(issue)]
    if location and 'All' not in location:
        filtered_df = filtered_df[filtered_df['Short_Location'].isin(location)]
    if month:
        filtered_df = filtered_df[filtered_df['Month'].isin(month)]

    if selected_cell and 'heatmap_id' in selected_cell:
        config = next((c for c in heatmap_configs if c['id'] == selected_cell['heatmap_id']), None)
        if config:
            x_axis = config['x_axis']
            y_axis = config['y_axis']
            filtered_df = filtered_df[(filtered_df[x_axis] == selected_cell['x']) & (filtered_df[y_axis] == selected_cell['y'])]

    return dcc.send_data_frame(filtered_df.to_csv, "filtered_incidents.csv", index=False)

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)